In [33]:
import os
import numpy as np
import bson

def getBsonFiles(DIR):
    files = os.listdir(DIR)
    bson_files = []

    for f in files:
        if ".bson" in f:
            bson_files.append(f)

    print(bson_files)
    return bson_files

In [34]:
DATA_DIR = os.getcwd() + "/data/fddg-db1"

In [35]:
bsongetBsonFiles(DATA_DIR)

['users.bson', 'metas.bson', 'chunks.bson']


['users.bson', 'metas.bson', 'chunks.bson']

In [32]:
bson_data = []

for b in bson_files:
    with open(DATA_DIR + "/" + b, "rb") as f:
        data = bson.decode_all(f.read())
        bson_data.append(data)
        #print(data)
    
print(bson_data[1])

[{'_id': '3520D804-0295-4978-90FE-193DCF83534C', 'subject_id': '3', 'phone_placement': 'right trouser pocket', 'recording_duration': -1, 'chunk_ids': ['CE06941E-8FFF-4DD5-B4F4-4E761CCFF4DB', '98DC445E-CE6C-4CE5-B5B9-562A157EC848', 'C7381008-6F5E-49B0-AFEA-3B7504389F9C', '3A7128ED-1012-42CA-A96C-67066ED1C28F', 'F2A3EB8B-0132-43EB-8E92-BC878C294FF1', 'AD7CB717-D192-4952-BFD5-D41F03331CC0'], '__v': 0}, {'_id': 'C001D50E-3A62-4516-9F61-9C1573932816', 'subject_id': '3', 'phone_placement': 'left trouser pocket', 'recording_duration': -1, 'chunk_ids': ['82EA548E-8D95-4A99-BD88-AB6609D166E1', '4A938083-C346-4852-98C2-4B848A269046', 'F7B53EEA-F615-4C66-A779-C5A10EBFB299', '521DB9A8-863C-4147-9688-1815BD84BE39', '2FF7573C-B929-46D5-88B3-0C3DF037EBF1', '802CEBF0-8C47-4C70-A812-E01047BA2F39', '74274907-217B-4148-8968-044ADC36DB9B', 'C80BC870-48EB-44C8-BBE0-5F853994D6F7'], '__v': 0}, {'_id': '2D938312-B18B-41BA-8DB2-92FE535B9E97', 'subject_id': '3', 'phone_placement': 'left trouser pocket', 'record